# Dernek bilgilerinin cekilmesi

Turkiye'de kayitli derneklerin bilgilerini cekmek icin bir Python 3 scriptini adim adim anlatiyorum. Veriler icin kaynagimiz Derbis (https://derbis.dernekler.gov.tr/) sistemi.

Python'un kendi kutuphanelerinin yanisira, HTTP request'lerini kolay yapabilmek icin `requests` kutuphanesini kullanacagiz. Eger ortaminizda kurulu degil ise kuralim.

In [ ]:
pip install requests

Kullanacagimiz kutuphaneleri import edelim. 

In [10]:
import json
import csv
import requests

Dernek bilgilerini il bazinda cekip, bir `.csv` dosyasina yazacagiz. Asagidaki fonksiyon, plaka numarasi verilen bir ildeki dernek bilgilerini cekip verdigimiz dosyaya ekleyecek. 

In [11]:
def get_il(il, csv_writer):
    data = json.dumps({
        'secilenTeskilatPk': il,
        'secilenIlcePk': '999999999',
        'neviler': '0',
        'altneviler': '0',
    })
    p = 1
    while True:
        response = requests.post(
            'https://derbis.dernekler.gov.tr/IstatistikDerneklerWeb/GetIlFaaliyetDernek',
            data={
                'page': p,
                'pageSize': 1000,
                'serializedData': data,
            },
            verify=False,
        )
        if reponse.json()['Errors']:
            # hata olmus; yazalim ve donguden cikalim
            print(reponse.json()['Errors'])
            break
        for dernek in response.json()['Data']:
            row = [
                dernek['Ad'],
                dernek['Sayi'],
                dernek['Oran'],
                dernek['Plaka'],
                dernek['AnaNevisi'],
                dernek['telNo'],
                dernek['Mail'],
                dernek['WebSitesi'],
                dernek['FkSisKodPkNevi'],
                dernek['FKSisTeskilatPk'],
                dernek['SisKod'],
                dernek['KurumAd'],
                dernek['KurumAdres'],
                dernek['Sira'],
                dernek['BaskanAdSoyad'],
                dernek['FkSisCityInfoPk'],
                dernek['AltNevisString'],
                dernek['IsActive'],
                dernek['DtOlusturma'],
                dernek['DtDegistirme'],
                dernek['Olusturan'],
                dernek['Degistiren'],
            ]
            csv_writer.writerow(row)
        if p * 1000 >= response.json()['Total']:
            # verilen il icin tum sayfalar cekildi
            break
        p += 1

Verileri kaydedecegimiz dosyayi olusturup kolon basliklarini yazdiralim. 

In [8]:
with open('dernekler.csv', 'w') as f:
    w = csv.writer(f)
    w.writerow([
        'Ad',
        'Sayi',
        'Oran',
        'Plaka',
        'AnaNevisi',
        'telNo',
        'Mail',
        'WebSitesi',
        'FkSisKodPkNevi',
        'FKSisTeskilatPk',
        'SisKod',
        'KurumAd',
        'KurumAdres',
        'Sira',
        'BaskanAdSoyad',
        'FkSisCityInfoPk',
        'AltNevisString',
        'IsActive',
        'DtOlusturma',
        'DtDegistirme',
        'Olusturan',
        'Degistiren',
    ])

Butun illerdeki derneklerin bilgilerini cekmek icin asagidaki donguyu kullanabilirsiniz.

In [ ]:
with open('dernekler.csv', 'a') as f:
    writer = csv.writer(f)
    for il in range(1, 82):
        get_il(il, writer)

Butun illere ihtiyaciniz yoksa veya cok vakit aldigi icin bolmeyi tercih ediyorsaniz, dongu kullanmadan illeri tek tek cekmeyi tercih edebilirsiniz. 

Burada anlatilan islemler, kaynaktaki degisikliklere gore guncelligini kaybedebilir. Bir hata fark ederseniz lutfen bana (onur [at] genelizleyici.com) bildirmekten cekinmeyin. 